In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('cldg').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/28 13:39:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/28 13:39:39 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
data = spark.read.csv('College.csv', inferSchema= True, header = True)

In [9]:
data.printSchema()

root
 |-- School: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)



In [10]:
data.head(1)

[Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60)]

In [11]:
data.columns

['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [12]:
from pyspark.ml.feature import VectorAssembler

In [13]:
assembler = VectorAssembler(inputCols=['Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate'], outputCol='features')

In [14]:
output = assembler.transform(data)

In [15]:
output.head(1)

23/12/28 13:46:53 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


[Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60, features=DenseVector([1660.0, 1232.0, 721.0, 23.0, 52.0, 2885.0, 537.0, 7440.0, 3300.0, 450.0, 2200.0, 70.0, 78.0, 18.1, 12.0, 7041.0, 60.0]))]

In [16]:
from pyspark.ml.feature import StringIndexer

In [17]:
index = StringIndexer(inputCol='Private', outputCol='privateIndex')

In [18]:
out_fixed = index.fit(output).transform(output)

In [19]:
final_data = out_fixed.select('features', 'privateIndex')

In [20]:
final_data.show()

+--------------------+------------+
|            features|privateIndex|
+--------------------+------------+
|[1660.0,1232.0,72...|         0.0|
|[2186.0,1924.0,51...|         0.0|
|[1428.0,1097.0,33...|         0.0|
|[417.0,349.0,137....|         0.0|
|[193.0,146.0,55.0...|         0.0|
|[587.0,479.0,158....|         0.0|
|[353.0,340.0,103....|         0.0|
|[1899.0,1720.0,48...|         0.0|
|[1038.0,839.0,227...|         0.0|
|[582.0,498.0,172....|         0.0|
|[1732.0,1425.0,47...|         0.0|
|[2652.0,1900.0,48...|         0.0|
|[1179.0,780.0,290...|         0.0|
|[1267.0,1080.0,38...|         0.0|
|[494.0,313.0,157....|         0.0|
|[1420.0,1093.0,22...|         0.0|
|[4302.0,992.0,418...|         0.0|
|[1216.0,908.0,423...|         0.0|
|[1130.0,704.0,322...|         0.0|
|[3540.0,2001.0,10...|         1.0|
+--------------------+------------+
only showing top 20 rows



In [24]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [25]:
from pyspark.ml.classification import (DecisionTreeClassifier, RandomForestClassifier, GBTClassifier)

In [44]:
dtc = DecisionTreeClassifier(featuresCol='features', labelCol='privateIndex')
rfc = RandomForestClassifier(numTrees=150, featuresCol='features', labelCol='privateIndex')
gbt = GBTClassifier(featuresCol='features', labelCol='privateIndex')

In [45]:
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [46]:
dtc_preds = dtc_model.transform(test_data)
rfc_preds = rfc_model.transform(test_data)
gbt_preds = gbt_model.transform(test_data)

In [47]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [48]:
my_bin_eval = BinaryClassificationEvaluator( labelCol='privateIndex')

In [49]:
print("DTC:",my_bin_eval.evaluate(dtc_preds))

DTC: 0.8850754310344828


In [50]:
print("RFC:",my_bin_eval.evaluate(rfc_preds))

RFC: 0.9854525862068966


In [51]:
print("GBT:",my_bin_eval.evaluate(gbt_preds))

GBT: 0.9483836206896543


In [52]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [53]:
my_eval_mul = MulticlassClassificationEvaluator(metricName='accuracy', labelCol='privateIndex')

In [55]:
print("dtc:acc:", my_bin_eval.evaluate(dtc_preds))

dtc:acc: 0.8850754310344828


In [57]:
print("rfc:acc:", my_bin_eval.evaluate(rfc_preds))

dtc:acc: 0.9854525862068966


In [58]:
print("gbt:acc:", my_bin_eval.evaluate(gbt_preds))

gbt:acc: 0.9483836206896543
